In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

filepath = "/Users/amanserhan/Desktop/Data Analysis Projects/Books Analysis/data/BooksDataset.csv"
data = pd.read_csv(filepath)

#Date column cleanup

data['Standard Date'] = pd.to_datetime(data['Publish Date'], errors='coerce')

date_format = "%B %Y"
data.loc[data['Standard Date'].isnull(), 'Standard Date'] = pd.to_datetime(
    data.loc[data['Standard Date'].isnull(), 'Publish Date'], 
    format=date_format, 
    errors='coerce'
)
cutoff_date = pd.to_datetime('2024-01-03')
data.loc[data['Standard Date'] > cutoff_date, 'Standard Date'] = pd.NaT

data['Year'] = data['Standard Date'].dt.year
data['Month'] = data['Standard Date'].dt.month

#print(data[data['Standard Date'].isnull()])

#Price column cleanup
data['Price']=data['Price'].str.slice(19)
data['Price'] = data['Price'].str.replace(r'[^\d.]', '', regex=True).astype(float)

#Category column cleanup 
data['Category'] = data['Category'].astype(str)
data['Genres_List'] = data['Category'].apply(lambda x: [genre.strip() for genre in x.split(',')])
all_genres = set()
for genres in data['Genres_List']:
    all_genres.update(genres)
genre_columns = pd.DataFrame()
genre_indicator_data = {}
for genre in all_genres:
    genre_indicator_data[genre] = data['Genres_List'].apply(lambda x: genre in x).astype(int)

genre_columns = pd.concat(genre_indicator_data, axis=1).reset_index(drop=True)

# Concatenate all lists in 'Genres_List' column into a single list
all_genres = sum(data['Genres_List'], [])

# Unique genres from the concatenated list
unique_genres = set(all_genres)

for genre in unique_genres:
    data[genre]=data['Genres_List'].apply(lambda x: genre in x).astype(int)




#features: month, year, genre, econ data
#target: price
#Read in econ data (recesssions)


column_names = data.columns.tolist()
print(column_names)
data.to_csv('test.csv', index=False)

/var/folders/7m/lvh1thjn04d16hnktt4_5qwr0000gn/T/ipykernel_4272/3467219780.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[genre]=data['Genres_List'].apply(lambda x: genre in x).astype(int)
/var/folders/7m/lvh1thjn04d16hnktt4_5qwr0000gn/T/ipykernel_4272/3467219780.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[genre]=data['Genres_List'].apply(lambda x: genre in x).astype(int)
/var/folders/7m/lvh1thjn04d16hnktt4_5qwr0000gn/T/ipykernel_4272/3467219780.py:52: PerformanceWarning: DataFrame is highly fragmented. 

['Title', 'Authors', 'Description', 'Category', 'Publisher', 'Publish Date', 'Price', 'Standard Date', 'Year', 'Month', 'Genres_List', 'Mountaineering', 'Clip Art', 'Growth', 'Assessment', 'Reggae', 'Environmental', 'Contemporary English Version', 'Forecasting', 'Criticism', 'Physical Impairments', 'College Guides', 'Media & the Law', 'Midwest (IA', 'Bottles', 'Ethnic & Regional', 'Pension Law', '17th Century', 'Reiki', 'Lesbian', 'Electricity & Electronics', 'Cartooning', 'People & Places', 'Cognitive Neuroscience & Cognitive Neuropsychology', 'Death', 'Social Theory', 'Country Life', 'Telepathy)', 'Gay Studies', 'Contemporary', 'Christian Education', 'Physical Therapy', 'Dye', 'Kayaking', 'Consulting', 'Kabbalah & Mysticism', 'Regional', 'Other', 'Emotions', 'Secondary', 'Interactive Adventures', 'CO', 'Music', 'Fantasy & Horror', 'Railroads & Trains', 'Orchids', 'Form', 'Camping', 'East South Central (AL', 'Strategy', 'Children', 'Passover', 'Environmental Science & Ecosystems', 'Dr

In [2]:
features = ['Year', 'Month', 'Poetry', 'Republic of South Africa', 'Koran & Sacred Writings', 'Tibetan', 'Visual BASIC', 'Health Risk Assessment', 'New Thought', 'Raw Food', 'Theater', 'Stewardship & Giving', 'Traditional', 'Cars & Trucks', 'Knowledge Capital', 'Auditing', 'Comparative Religion', 'Public Health', 'Linear', 'Urban Life', 'Sandwiches', 'Visionary & Metaphysical', 'Tablesetting', 'Birdwatching Guides', 'Meteorology & Climatology', 'Self-Esteem & Self-Reliance', 'Diseases', 'Desserts', 'Personal Finance Applications', 'Comparative', 'Mathematics', 'French', 'Afghan War (2001-)', 'Healing', 'Myths', 'Higher', 'Essays', 'Drawing', 'Printmaking & Stamping', 'Biochemistry', 'Pathology', 'Organizational Behavior', 'Individual Artists', 'Communication', 'Greek & Roman', 'Ancient & Classical', 'Facility Management', 'Rome', 'Comics & Graphic Novels', 'Sky Observation', 'Sustainable Development', 'Personality Disorders', 'Astrophysics', 'High Tech', 'Political Parties', 'Daily Activities', 'Adventure', 'Ornithology', 'Spelling & Vocabulary', 'Death & Dying', 'Mountain (AZ', 'Public Contract', 'Zoos', 'Humanism', 'Classical & Medieval', 'Boating', 'Regional', 'Running & Jogging', 'Mineralogy', 'Shakespeare', 'Pigs', 'Squirrels', 'Home Schooling', 'Russian & Former Soviet Union', 'Garden Furnishings', 'Caribbean & Latin American', 'Antique & Classic', 'Anesthesiology', 'Emotional & Social Disabilities', 'Book Printing & Binding', 'Self-Hypnosis', 'Lesbian', 'Cross-Stitch', 'Women & Relationships', 'Adoption & Fostering', 'Woodwinds', 'Differential', 'Middle Atlantic (NJ', 'Alcohol', 'Caregiving', 'Probability & Statistics', 'Literary Collections', 'Online Safety & Privacy', "Buyer's Guides", "International Children's Bible", 'Civil War Period (1850-1877)', 'Legal', 'Garden Design', 'Juvenile Nonfiction', 'Water Sports', 'Investing', 'Technical Writing', 'Cooking & Food', 'Diplomacy', 'Hiking', 'Pre-Calculus', 'Single Parent', 'India', 'North America', 'Commentary & Opinion', 'Natural Resources', 'Penology', 'Social Policy', 'Weight Loss', 'Family Law', 'Affirmations', 'Metaphysics', 'Needlework', 'Hard-Boiled', 'Careers', 'African', 'Japan', 'Elementary', 'Discrete Mathematics', 'Dough', 'Optical Illusions', 'Spiritual Growth', 'Hearing & Speech', 'Oncology', 'Oncology & Cancer', 'Middle Atlantic (DC', 'Addiction', 'Object Oriented', 'Domestic Partner Abuse', 'East South Central (AL', 'Social Services & Welfare', 'Caribbean & West Indies', 'Banks & Banking', 'Government & Business', 'Religion & Science', 'Holiday Music', 'Anatomy & Physiology', "Valentine's Day", 'Biblical Reference', 'Portraits & Selfies', 'Legislative Branch', 'Aviation', 'Experimental Methods', 'Criminal Law', 'Gymnastics', 'Prayerbooks', 'Figurines', 'Antiques & Collectibles', 'Exercise', 'Quality Control', 'Salads', 'A+', 'Americana', 'Future Studies', 'Jurisprudence', 'Travel & Tourism', 'Animals', 'Diet & Nutrition', 'Books', 'Critical & Intensive Care', 'Mammals', 'Life Stages', 'Public', 'Ethnic & Regional', 'Human Geography', 'Flowers', 'Taxonomy', 'Asian American', 'Herbs', 'Vegetarian & Vegan', 'Political Science', "Children's", 'Social Theory', 'Eating Disorders & Body Image', 'Primates', 'Reiki', 'Risk Assessment & Management', 'Christian Theology', "Children's & Young Adult Literature", 'Sauces & Dressings', 'Plants', 'Human Resources & Personnel Management', 'Literacy', 'Mergers & Acquisitions', 'Love & Romance', 'Aromatherapy', 'Methodist', 'System Theory', 'Sustainable Living', 'Hymns', 'Nursing Home Care', 'Catholic', 'Appetizers', 'Globalization', 'Atheism', 'Musicals', 'Personal Finance', 'Treaties', 'Entomology', 'Young Adult Fiction', 'Christian', 'Naval', 'Clocks & Watches', 'Church of Jesus Christ of Latter-day Saints (Mormon)', 'Nature & the Natural World', 'Divination', 'Chemotherapy', 'Intellectual Property', 'Friendship', 'Chemistry', 'Cleaning', 'Exports & Imports', 'Acoustics & Sound', 'Exegesis & Hermeneutics', 'Acupressure & Acupuncture', 'Gardening', 'Testing & Measurement', 'Environment', 'Substance Abuse', 'Local Area Networks (LANs)', 'Asian American Studies', 'Individual Director', 'Multigenerational', 'Computers & Technology', 'Electricity & Electronics', 'Mind & Spirit', 'Wizards & Witches', 'Direction & Production', 'German', 'Skateboarding', 'Movements', 'Language Arts & Disciplines', 'Pottery & Ceramics', 'Personal Growth', 'Internships', 'Security (National & International)', 'Women', 'Switzerland', 'Catalogs', 'New Baby', 'Psychiatric', 'Wolves & Coyotes', 'Cats', 'Video & Animation', 'Turtles', 'Pollution Control', 'White Collar Crime', 'Language Arts', 'Caretaking & Organizing', "New International Reader's Version", 'Apologetics', 'Specific Ingredients', 'Embroidery', 'Baptist', 'Hematology', 'WY)', 'IN', 'Body', 'Comic Strips & Cartoons', 'Physiology', 'Places', 'Continental European', 'CAD-CAM', 'New England', 'Email Clients', 'Science Fiction & Fantasy', 'Beauty & Grooming', 'Green Business', 'Yiddish', 'Pathophysiology', 'Men', 'Sign Language', 'Persian Gulf War (1991)', 'Beadwork', 'New Testament', 'Hypnotism', 'Lasers & Photonics', 'Grammar & Punctuation', 'Individual Composer & Musician', 'Evangelism', 'Ventilation & Air Conditioning', 'Collections', 'Legal Writing', 'Alternative History', 'Skin', 'Fantasy & Magic', 'Individual Photographers', 'Butterflies & Moths', 'Drafting & Mechanical Drawing', 'Non-Euclidean', 'Small Animal', 'Political Ideologies', 'Repair & Maintenance', 'Amphibians & Terrariums', 'Industries', 'Databases', 'Legends', 'Casseroles', 'Potpourri', 'Defamation', 'Messianic Judaism', 'Educational Law & Legislation', 'Calligraphy', 'Regional Planning', 'Appliances & Supplies', 'Country & Bluegrass', 'Gold & Other Metals', 'Peer Pressure', 'Russian', 'Courses & Dishes', 'School Age', 'Study & Teaching', 'C', 'Americas (North', 'Illnesses & Injuries', 'Baby Names', 'Textile & Costume', 'Lace & Tatting', 'Morocco', 'Former Soviet Republics', 'Meat', 'Codependency', 'Chocolate', 'Time Travel', 'Management', 'Landscapes & Seascapes', 'Ecosystems & Habitats', 'Gluten-Free', 'Counting & Numeration', 'Evaluation & Assessment', 'Bible Study Guides', 'Dance', 'Stagecraft & Scenography', 'Italian', 'Training', 'Public Finance', 'Herbal Medications', 'Management & Leadership', 'Plastic & Cosmetic', 'Spanish & Portuguese', 'Equipment & Supplies', 'Porcelain & China', 'Police Procedural', 'Sewing', 'Satire', 'Teaching Methods & Materials', 'Quilts & Quilting', 'Eastern', 'Russia & the Former Soviet Union', 'Christian Education', 'Country & Ethnic', 'Volleyball', 'Living Trusts', 'Macroeconomics', 'Winter Sports', 'West South Central (AR', 'Industrial', 'Macrobiotics', 'Diabetes', 'Crisis Management', 'Ice Skating', 'Military & Wars', 'Supernatural', 'Clinical Medicine', 'Zoology', 'Anthologies', 'Classroom Management', 'Emigration & Immigration', 'Outdoor', 'Lodging & Transportation', 'Administrative Law & Regulatory Practice', 'Money & Monetary Policy', 'Travel Games', 'Exhibitions', 'Healthy Living', 'Law Enforcement', 'Scandinavian Languages (Other)', 'etc.)', 'Right to Die', 'Fashion & Beauty', 'Urban', 'Rock', 'Sales & Selling', 'Fish & Aquariums', 'Analytic', 'Sagas', 'Counseling', 'Soul Food', 'Women Sleuths', 'Text', 'Consumer Guides', 'Rabbits', 'Sports Medicine', 'Reading Skills', 'Guides & Reviews', 'Scottish', 'Ireland', 'Cookery for One', 'Practical Guides', 'Test Preparation', 'Architecture', 'Interior Design', 'Insects & Spiders', 'Ethnic & Tribal', 'First Aid', 'Shamanism', 'Commercial & Corporate', 'Manners & Etiquette', 'Anger', 'Family & Relationships', 'Cajun & Creole', 'Tours', 'Museum Administration & Museology', 'Business Law', 'Gothic & Romance', 'Atlases & Gazetteers', 'Autism Spectrum Disorders', 'Kayaking', 'Emotions', 'Contracting', 'Communication Studies', 'FL', 'Intranets & Extranets', 'Oceans & Seas', 'Health Policy', 'Commerce', 'Appreciation', 'Prehistory', 'Oceanography', "Children's Music", 'Film', 'Leadership & Mentoring', 'Outdoor & Recreational Areas', 'Gay', 'Faith', 'Romantic Comedy', 'Darkroom', 'Germany', 'NV', 'Canada', 'Infinity', 'Imagination & Play', 'Bibliographies & Indexes', 'Medical & Surgical', 'Philosophy & Social Aspects', 'Saints & Sainthood', 'New American Bible', 'Decision-Making & Problem Solving', 'Biblical Meditations', 'Marriage & Long-Term Relationships', 'Regional Studies', 'Genealogy & Heraldry', 'Political Freedom & Security', 'Personal Computers', 'Trees & Forests', 'Mermaids', 'Occult & Supernatural', 'Buddhism', 'Biography & Autobiography', 'Attention Deficit Disorder (ADD-ADHD)', 'Certification Guides', 'Marriage & Family', 'Physical & Emotional Abuse', 'Boys & Men', 'etc.', 'Benelux Countries (Belgium', 'Political Advocacy', 'Financial', 'Financial Aid', 'Document Management', 'Tobacco-Related', 'Seasons', 'NC', 'Collaboration Software', 'Nationalism & Patriotism', 'Biographical', 'New Living Translation', 'Energy', 'Basketball', 'Geology', 'Sweden)', 'Prejudice & Racism', 'Sleep & Sleep Disorders', 'Christian Life', 'Data Transmission Systems', 'Managerial', 'Allergy', 'MI', 'Orchids', 'Witchcraft', 'Manufacturing', 'Early Childhood (incl. Preschool & Kindergarten)', 'Canadian', 'Needlepoint', 'Hockey', 'SQL', 'Digital', 'Air Sports', 'Bibles', 'AR', '18th Century', 'Enterprise Applications', 'Hinduism', 'Limericks & Verse', 'Conducting', 'Humanistic', 'Mood Disorders', 'Aerobics', 'Magic', 'International', "Discography & Buyer's Guides", 'Telemarketing', 'Amish & Mennonite', 'Medieval', 'Equestrian', 'Client-Server Computing', 'Walking', 'Chaotic Behavior in Systems', 'Entertaining', 'China', 'House Plants & Indoor', '17th Century', 'Contemporary', 'Soccer', 'Applied', 'Clip Art', 'Science & Nature', 'Alphabet', 'Gerontology', 'Vegetarian', 'Physicians', 'Endocrinology & Metabolism', 'Microeconomics', 'Siblings', "Alzheimer's & Dementia", 'Allied Health Services', 'Diet Therapy', 'Graduate School Guides', 'Apple (Mac OS & iOS)', 'Labor & Industrial Relations', 'Card Games', 'Civil Law', 'MS', 'Assessment & Diagnosis', 'Fishes', 'Government', 'Criminology', 'Political Freedom', 'Computers', 'Corporate & Business History', 'Bipolar Disorder', 'Students & Student Life', 'Games', 'Publishing', 'Computer Engineering', 'Essays & Travelogues', 'Inspirational', 'Applied Sciences', 'Emotions & Feelings', 'Professional Development', 'Literary Criticism & Collections', 'International Security', 'Farm Animals', 'Native American', 'Pictorials', 'Television', 'Theory', 'Social Work', 'Sacred Writings', 'Mathematical Analysis', 'Study & Test-Taking Skills', 'Diagnosis', 'Martial Arts & Self-Defense', 'Dictionaries & Terminology', 'Child Abuse', 'Research & Theory', 'Women Authors', 'Calculus', 'Boats', 'Residential', 'NON-CLASSIFIABLE', 'Test Preparation & Review', 'Maintenance & Repair', 'Reproductive Medicine & Technology', 'WV)', 'Food', 'Points of Interest', 'Violence', 'Veterinary Medicine', 'College Entrance', 'Canning & Preserving', 'Lasers in Medicine', 'Organizations & Institutions', 'Midwest (IA', 'Furniture', 'Letters', 'Physical Impairments', 'Buildings', 'Decorative Arts', 'Presentation Software', 'Horror & Ghost Stories', 'Hikes & Walks', 'Taxation', 'Fashion', 'Military', 'Diabetic & Sugar-Free', 'Optics & Light', 'Carpentry', 'LA', 'Hotels', 'Accounting', 'Psychopharmacology', 'Censorship', 'Homeopathy', 'Neuropsychology', 'Ornamental Plants', 'Encyclopedias', 'Dolls', 'Industrial Management', 'Economic Development', 'Dating', 'Economic Conditions', 'Human Sexuality', 'Lesbian Studies', 'Drama', 'Women Artists', 'Fiction', 'Bowling', 'Techniques', 'Foreign Language Study', 'Mashups', 'Communication & Social Skills', 'Reptiles', 'Quantity', 'Social Science', 'Weapons', 'Authorship', 'ACT', 'Remodeling & Renovation', 'Trial Practice', 'Online Services', 'Molecular Biology', 'Coyotes & Wild Dogs', 'Aeronautics & Astronautics', 'Dating & Relationships', 'Consumer Behavior', 'OR', 'Gastrointestinal', 'Annuals', 'Jazz', 'Renaissance', 'Transcendentalism', 'Production & Operations Management', 'Discrimination', 'Power Tools', 'Fisheries & Aquaculture', 'Electrical', 'Models', 'Biostatistics', 'City & Town Life', 'Oil Painting', 'History', 'Clergy', 'Stories in Verse', 'New England (CT', 'Electronic Publishing', 'MN', 'Slavery', 'Programming Languages', 'History & Culture', 'Seashells', 'Negotiating', 'Physical', 'Design & Construction', 'Outdoor Skills', 'UNIX', 'Motivational', 'Creativity', 'Pastoral Resources', 'Bankruptcy & Insolvency', 'Ceramics', 'Fish', 'Hunting', 'Twelve-Step Programs', 'Finance', 'Russia', 'English Standard Version', 'Business Writing', 'Equipment', 'Europe', 'Bodybuilding & Weight Training', 'Israel', 'Agriculture & Food', 'Special Education', 'Worship & Liturgy', 'Crime', 'NS', 'Historiography', 'Heating', 'Foreign Exchange', 'Educators', 'Audiology & Speech Pathology', 'Cognitive Psychology & Cognition', 'Italy', 'CD-DVD Technology', 'Legal History', 'Thrillers', 'Study', 'Desktop Applications', 'Study Skills', 'Hebrew', 'Learning Disabilities', 'Nanotechnology & MEMS', 'Nursing', 'Civics & Citizenship', 'Thai', 'E-Commerce', 'Great Britain', 'Sleep', 'Questions & Answers', 'World', 'Technology & Engineering', 'West (AK', 'NE', 'Tests (Other)', 'Baking', 'Weight Control', 'Respiratory', 'Adult & Continuing Education', 'Money Management', 'Football', 'Social Psychology', 'Religious', 'Royalty', 'Drug Guides', 'Library & Information Science', 'GA', 'Nature', 'Yearbooks & Annuals', 'Southwest', 'Distribution', 'Urology', 'Purchasing & Buying', 'Topiary', 'Country Life', 'Nurse & Patient', 'Pulmonary & Thoracic Medicine', 'Vegetables', 'Voice', 'Heart', 'Martial Arts', 'Secondary', 'Business Aspects', 'Gourmet', 'Art & Politics', 'Adoption', 'Circuits', 'Stenciling', 'Feng Shui', 'Mexican', 'Spain & Portugal', 'Neurology', 'Illustration', 'Political Economy', 'Behavioral', 'Beverages', 'Antitrust', 'Gay Studies', 'Field Hockey', 'Low Salt', 'Precognition', 'Color Theory', 'Environmental Science & Ecosystems', 'Assessment', 'Book', 'Student Life & Student Affairs', 'New King James Version', 'Caribbean & Latin America', 'Anglican', 'Weather', 'Urban & Regional', 'Physiological Psychology', 'Media & the Law', 'Historical', 'Adolescence', 'State & Provincial', 'National', 'Carving', 'Jewish', 'Alcoholic', 'GRE (Graduate Record Examination)', 'Life Sciences', 'Automobile Industry', 'Ethnic Studies', 'Developmental', 'Editors', 'Dynamics', 'Alternative Family', "Children's Literature", 'Presbyterian', 'Executive Branch', 'Psychology', 'Commercial Policy', 'Words', 'African American Studies', 'Occultism', 'Scottish & Welsh', 'Customs & Traditions', 'Prehistoric', 'American Government', 'Ancient Mysteries & Controversial Knowledge', 'Genetics & Genomics', 'Dutch', 'Pension Law', 'Individual Architects & Firms', 'New Experience', 'Atomic & Molecular', 'Counseling & Recovery', 'Disasters', 'Public Policy', 'JavaScript', 'Browsers', 'Opera', 'Language Study', 'Home-Based Businesses', 'Spirituality', 'Woodwork', 'Fables', 'Pharmacy', 'History & Theory', 'Northwestern States', 'Cardiology', 'Mediterranean', 'Network Protocols', 'History & Philosophy', 'Shade', 'OK', 'Reincarnation', 'Astrology', 'Media & Communications', 'Sustainability & Green Design', 'Chapter Books', 'Tennis', 'General', 'Politics & State', 'Reform', 'Performing Arts', 'Ethics & Professional Responsibility', 'Social Activists', 'Literary Figures', 'Family & General Practice', 'Archaeology', 'GMAT (Graduate Management Admission Test)', 'Middle Western States', 'Oceania', 'Neuroscience', 'VT)', 'Forensic Science', 'War & Military', 'Econometrics', 'Kitchenware', 'Perennials', 'Native American Studies', 'Ghost Stories', 'Statistics', 'Leopards', 'Security', 'Etymology', 'Pascal', 'Rural', 'Political Process', 'Windows Desktop', 'Corporate', 'Hanukkah', 'Software', 'Patriotic Holidays', 'Drug Dependence', 'Political', 'Video & Electronic', 'Sculpture', 'Rivers', 'Occupational Therapy', 'Construction', 'Meditation', 'House & Home', 'Concepts', 'ND', 'Health Care Delivery', 'Wine', 'Baltic States', 'Bread', 'Internal Medicine', 'Golf', 'Physical Medicine & Rehabilitation', 'Colonial Period (1600-1775)', 'Mathematical & Statistical Software', 'Sounds', 'Pen & Ink Drawing', 'Crafts & Hobbies', 'Arabic', 'Paleontology', 'PE)', 'Form', 'Business Development', 'East North Central (IL', 'Children & Youth', 'Science Fiction', 'Dictionaries', 'Anthropology', 'Jungian', 'Post-Communism & Socialism', 'Business Etiquette', 'Songwriting', 'Social Topics', 'Histology', 'WA)', 'United States', 'Business & Professional', 'Native American & Aboriginal', 'Austria', 'Professional Growth', 'Civil', 'Industrial Technology', 'Textiles & Costume', 'Business Communication', 'Soups & Stews', 'Word & Word Search', 'NM', 'Jewish Studies', 'Korean War', 'Drugs', 'Inventions', 'DE', 'Terrorism', 'Japanese', 'Aeronautics', 'Episcopalian', 'Hamsters', 'Pictorial', 'Academic Development', 'Stress Management', 'Corporate Finance', 'Austria & Hungary', 'Special Needs', 'Regency', 'BC)', 'Fruit', 'Australia & New Zealand', 'Celebrity', 'Systems Architecture', 'Social Issues', 'Freemasonry', 'Petroleum', 'New Revised Standard Version', "Women's Issues", 'Bridges', 'Road Travel', 'Cooking', 'International Mystery & Crime', 'Computer Graphics', 'Seafood', 'Television & Video', 'Traditions', 'Relativity', 'Israel & Palestine', 'Insurance', 'Clinical Psychology', 'Crocheting', 'Psychoanalysis', 'Far East', 'New International Version', 'Industrial Engineering', 'Bookkeeping', 'Evidence', 'Entrepreneurship', 'Local', 'Ethnomusicology', 'SD', 'Bulbs', 'Exploration & Discovery', 'Marine Biology', 'Interpersonal Relations', 'Girls & Women', 'Criminal Procedure', 'Activities', 'LGBT Studies', 'Data Processing', 'Nuclear Warfare', 'Cinematography & Videography', 'Games & Activities', 'Automotive', 'Cancer', 'Composers & Musicians', 'Parks & Campgrounds', 'Human Rights', 'Artists', 'Curiosities & Wonders', 'Biblical Biography', 'West', 'PA)', 'Cacti & Succulents', 'Marriage & Divorce', 'Emergency Medicine', 'Personal & Practical Guides', 'Hospitality', 'UFOs & Extraterrestrials', 'Fiction Writing', 'How Things Work-Are Made', 'Latin', 'Minority Studies', 'Health & Daily Living', 'Linux', 'Search Engines', 'True Crime', 'Bridge', 'Personality', 'Pies', 'Horoscopes', 'Musical Instruments', 'Insects', 'College Guides', 'Farm & Ranch Life', 'Peace', 'Occupations', 'Diagnostic Imaging', 'Advertising', 'Matrices', 'Taoist', 'Landscape', 'Speeches', 'Sociolinguistics', 'Old Testament', 'Family', 'Study Guides', 'Schizophrenia', 'Landscapes', 'American', 'Cookies', 'Naturopathy', 'Anecdotes', 'Mechanics', 'Ducks', 'Time Management', 'Nonfiction (incl. Memoirs)', 'Dinosaurs & Prehistoric Creatures', 'Theater & Musicals', 'Multi-Language Dictionaries', 'Nature Crafts', 'Australia & Oceania', 'Politics & Government', 'Fairy Tales & Folklore', 'Stepfamilies', 'Strangers', 'Animal Husbandry', 'Gothic', 'Chess', 'Africa', 'Folklore & Mythology', 'Internet', 'Collections & Anthologies', 'Preschool & Kindergarten', '20th Century', 'Chronic Fatigue Syndrome', 'Finite Mathematics', 'Indian & South Asian', 'Ancient Civilizations', 'Missions', 'Photographers', 'Brunch', 'Semantics', 'Middle Atlantic States', 'Holidays (non-religious)', 'Acupuncture', 'Iraq War (2003-2011)', 'Film & Video', 'Papercrafts', 'Mind & Body', 'Abortion & Birth Control', 'Drawing & Presentation', 'Inspiration & Personal Growth', 'Study Aids', 'Central American & South American', 'Social Scientists & Psychologists', 'Asia', 'Fantasy', 'Marketing', 'Puzzles', 'Parapsychology', 'Site Directories', 'Values & Virtues', 'Runaways', 'Abuse', 'House Plans', 'Sexual Instruction', 'Child', 'AIDS & HIV', 'Biotechnology', 'Special Forces', 'Substance Abuse & Addictions', 'Table', 'Style Manuals', 'City Planning & Urban Development', 'Business Ethics', 'Systematic', 'Medical', 'Luxembourg)', 'Love & Erotica', 'South (AL', 'Military Science', 'Painting', 'the Gospels & Acts', 'Bottles', 'Law & Crime', 'MCAT (Medical College Admission Test)', 'Non-Formal Education', 'Conceptual', 'Health Care Issues', 'Gender Studies', 'Civil Rights', 'Research', 'Books & Reading', 'Sexual Abuse', 'Birthdays', 'Popular Culture', 'Holism', 'Manga', 'Landmarks & Monuments', 'Garnishing', 'Computers & Information Technology', 'Glass & Glassware', 'Trees', 'Self-Management', 'Low Fat', 'Rich & Famous', 'Printed Music', 'Social Classes & Economic Disparity', 'Flowers & Plants', 'Practice Management & Reimbursement', 'Currency & Medals', 'Circus', 'Earthquakes & Volcanoes', 'Memory Improvement', 'Judicial Power', 'Lawyers & Judges', 'Methods', 'Activity Books', 'Gastroenterology', 'Network Hardware', 'Service', 'Literary Criticism', 'World War II', 'Christian Ministry', 'Guitar', 'Astronomy', 'Constitutions', 'Gazetteers & Maps', 'Economics', 'Sikhism', 'Industrial Health & Safety', 'Web', 'Journaling', 'Colonial & Revolutionary Periods', 'Anatomy', 'Directories', 'Special Interest', 'Poultry', 'Parents', 'Mental Health', 'Logic & Brain Teasers', 'Aging', 'Norway', 'Foxes', 'Advertising & Promotion', 'Latin America', 'Essays & Writings', 'Breakfast', 'Peripherals', 'Sports & Recreation', 'Self-Esteem', "Paul's Letters", 'Parent & Adult Child', 'Electronics', 'Ships & Underwater Craft', "God's Word", 'Cartooning', 'Cruises', 'Music', 'Criminals & Outlaws', 'Vitamins', 'Veterans', 'Fashion & Accessories', 'Terminal Care', 'Blackjack', 'Fairy Tales', 'New Testament & Portions', 'Human Anatomy & Physiology', 'Existentialism', 'Biblical Commentary', 'Weddings', 'Business', 'Earth Sciences', 'Quotations', 'Drafting', 'Health & Healing', 'Classical', 'Bereavement', 'Board Games', 'Radio', 'Nonprofit Organizations & Charities', 'Modern', 'Youth', 'Blended Families', 'Dentistry', 'Serial Killers', 'Civilization', 'Systems Analysis & Design', 'Fundamentals & Skills', 'WI)', 'Art & Architecture', 'Boarding School & Prep School', 'Intermediate', 'Linguistics', 'Subjects & Themes', "Men's Studies", 'BASIC', 'Ships & Shipbuilding', 'Cakes', 'Publishers & Publishing Industry', 'Psychiatry', 'Energy (Qigong', 'Ghost', 'MO', 'Motherhood', 'Dictionaries & Encyclopedias', 'Sexuality', 'RI', 'Classics', 'Short Stories (single author)', 'Christian Rituals & Practice', 'Songbooks', 'Murder', 'Classical & Ballet', 'Constitutional', 'Dairy', 'Anger Management', 'Nervous System (incl. Brain)', 'Poverty & Homelessness', 'Psychological', 'Fantasy Sports', 'Middle East', 'Organizational Development', 'Structural Adjustment', 'Legal Education', 'Parodies', 'Strategy', 'European', 'Fatherhood', 'Bartending', 'Book Notes', 'Photography', 'Acrylic Painting', 'Do-It-Yourself', 'Passover', 'Theory & Social Aspects', 'Safety', 'Multicultural Education', 'Piano & Keyboard', 'Pain Management', 'Reading & Phonics', 'Feminist', 'Legends & Mythology', 'Photoessays & Documentaries', 'Eschatology', 'Stuffed Animals', 'Size & Shape', 'Beginner', 'Courts', 'Broadway & Musicals', 'Organic', 'I Ching', 'Restaurants', 'Camping', 'Composition', 'Bedtime & Dreams', 'Hispanic American Studies', 'State', 'Westerns', 'Confectionery', 'Wine & Spirits', 'Nonfiction', 'Liberation', 'Audio', 'Swimming', 'Oral Health', 'Silver', 'Animal Rights', 'Hispanic American', 'Computers & Digital Media', 'Arms Control', 'Orthodox', 'Women in Politics', 'Multiple Translations', 'Southwestern States', 'Young Adult Nonfiction', 'Massage & Reflexotherapy', '19th Century', 'Screenwriting', 'Consulting', 'Water (Oceans', 'Piloting & Flight Instruction', 'Networking', 'Oral & Maxillofacial', 'Non-Alcoholic', 'Reptiles & Amphibians', 'Central America', 'Southeast Asia', 'Happiness', 'Playwriting', 'Speech & Pronunciation', 'Other', 'Property', 'Sports', 'Alien Contact', 'Exercises', 'Theology', 'Public Speaking', 'Acting & Auditioning', 'Central & South America', 'Waves & Wave Mechanics', 'Philosophers', 'Software Development & Engineering', 'Quick & Easy', 'Handbooks', 'Maps & Road Atlases', 'Natural Disasters', 'Jokes & Riddles', 'Humorous Stories', 'Cultural Heritage', 'Lions', 'Dreams', 'Cultural & Social', 'Environmental Economics', 'Netherlands', 'India & South Asia', 'Success', 'Children with Special Needs', 'Windows Server', 'NL', 'Iceland & Greenland', 'Cognitive Psychology', 'Surgery', 'Resorts & Spas', 'Guinea Pigs', 'Shooting', 'Personal Memoirs', 'Telecommunications', 'Printmaking', 'Inspirational & Religious', 'Plants & Animals', 'Sculpture & Installation', 'Semiotics & Theory', 'Day Care & Child Care', 'Journalists', 'Court Rules', 'Business & Financial', 'NY', 'Total Quality Management', 'Adolescence & Coming of Age', 'Olympics & Paralympics', 'Common English Bible', 'Resumes', 'South America', 'Lifespan Development', 'Maturing', 'Culinary', 'Instruction & Study', 'Torts', 'Developing & Emerging Countries', 'Health & Fitness', 'Chiropractic', 'Photojournalism', 'Budgeting', 'Cosmology', 'Personal Success', 'Cultural Policy', 'Meditations', 'NGOs (Non-Governmental Organizations)', 'Vietnam War', 'Human-Computer Interaction (HCI)', 'Toilet Training', 'Antiquities & Archaeology', 'Health', 'Marine Life', 'Interactive Adventures', 'Social Security', 'Estates & Trusts', 'Southwest (AZ', 'Cycling', 'Wills', 'Mixed Media', 'Ethics & Moral Philosophy', 'Seasonal', 'Business & Economics', 'Gynecology & Obstetrics', 'Prayer', 'Recreations & Games', 'Educational Psychology', 'Racquetball', 'Patchwork', 'Historical Geography', 'Horror', 'Paranormal', 'Baseball & Softball', 'Adult Children of Substance Abusers', 'Agnosticism', 'Pentecostal & Charismatic', 'Color', 'Self-help', 'Confucianism', 'Creative Ability', 'Suicide', 'Islam', 'Labor', 'SC', 'Alternative Therapies', "Men's Interests", 'Welsh', 'Transportation', 'Methodology', 'Machinery & Tools', 'Folk & Outsider Art', 'Survival', 'Sociology of Sports', 'Sociology', 'Senses & Sensation', 'Labor & Employment', 'Southern', 'Physics', 'Sociology of Religion', 'Thanksgiving', 'Pets', 'English as a Second Language', 'Computer Science', "Women's Interests", 'Project Management', 'Fossils', 'Office Management', 'Western Provinces (AB', 'Critical Care', 'Natural Foods', 'Pirates', "Jehovah's Witnesses", 'History & Surveys', 'Architects', 'CO', 'Small Business', 'Pastel Drawing', 'Supernatural (incl. Ghosts)', 'Imaging Technologies', 'Digital Media', 'ME', 'Multilevel', 'Bible Stories', 'Northeast', 'Adventurers & Explorers', 'Embryology', 'Flower Arranging', 'Commercial & Industrial', 'Reggae', 'Garnishing & Food Presentation', 'UT', 'Museum Studies', 'Database Management', 'Engineering (General)', 'Handwriting Analysis', 'Criticism & Theory', 'Physician & Patient', 'Administration', 'Customer Relations', 'Public Speaking & Speech Writing', 'Concordances', 'Orphans & Foster Homes', 'Birds & Birdwatching', 'Etiquette', 'Espionage', 'Mindfulness & Meditation', 'Forensic Psychology', 'Counting & Numbers', 'Extreme Sports', 'Curricula', 'Growth', 'Workplace Culture', 'Real Estate', 'Scandinavia (Finland', "Women's Studies", 'Screenplays', 'Parasitology', 'People with Disabilities', 'Microbiology', 'Design', 'Economic Policy', 'TX)', "Men's Issues", 'Perinatal', 'Special Appliances', 'People & Places', 'Quantum Theory', 'Epidemiology', 'Superheroes', 'Sacraments', 'Experiments & Projects', 'Romance', 'English', 'Spiritual', 'Teddy Bears', 'Art', 'Fantasy & Horror', 'Agriculture', 'Sermons', 'Applied Psychology', 'Legal Services', 'Greece', 'Angelology & Demonology', 'Coming of Age', 'Retailing', 'Ecuador & Galapagos Islands', 'Marriage', 'Research & Methodology', 'Senior', 'Central', 'Geophysics', 'Remote Sensing & Geographic Information Systems', 'Mennonite', 'Investments & Securities', 'Motivational & Inspirational', 'Telepathy)', 'Information Technology', "Women's Health", 'Coaching', 'Condiments', 'Spreadsheets', 'Career Development', 'Container', 'Cults', 'Rocks & Minerals', 'Applique', 'Divorce & Separation', 'Essays & Narratives', 'Evolution', 'Australian & Oceanian', 'Swimming & Diving', 'Contemporary English Version', 'Hispanic & Latino', 'African American', 'Poker', 'Soteriology', 'Virtual Worlds', 'Pop Vocal', 'Folk Tales', 'Social Networking', 'Lacrosse', 'Mountaineering', 'Cultural', 'Hand Tools', 'Interactive & Multimedia', 'Viruses & Malware', 'Toymaking', 'SD)', 'Propaganda', 'Barbecue & Grilling', 'Caribbean & West Indian', 'Pragmatism', 'Orthopedics', 'Lawns', 'Free Will & Determinism', 'Monkeys', 'Science & Technology', 'Word Games', 'Preventive Medicine', 'KY', 'Education', 'Other Translations', 'Atlantic Provinces (NB', 'Homelessness & Poverty', 'Canoeing', 'MT', 'Logic', 'Algebra', 'History & Criticism', 'Motor Sports', 'Laboratory Medicine', 'Pediatric & Neonatal', 'Medical History & Records', 'Dating & Sex', 'Track & Field', 'Magical Realism', 'Ballet', 'Democracy', 'Modern (late 19th Century to 1945)', 'Patent', 'Horse Racing', 'Spanish', 'C++', 'Mice', 'OH', 'Watercolor Painting', 'Trade & Tariffs', 'Electronic Commerce', 'Economic History', 'Windows Server & NT', 'Publishers', 'Leadership', 'Urban & Land Use Planning', 'Adult', 'Business Mathematics', 'Power Resources', 'Physical Education', 'Readers', 'Advanced Placement', 'Unexplained Phenomena', 'Pharmacology', 'Social History', 'Puppets', 'Asian', 'Macintosh', 'Psychology of Religion', 'Site Design', 'Religion', 'MA', 'Discrimination & Race Relations', 'Decorating', 'Media Tie-In', 'Judicial Branch', 'Mathematical & Computational', 'Criticism', 'nan', 'Windows Workstation', 'Mexico', 'LGBT', "Children's Studies", 'Pastry', 'Geriatrics', "Men's Health", 'Development', 'Teenagers', 'Aerial', 'Massage & Reflexology', 'Revolutionary Period (1775-1800)', 'Angels & Spirit Guides', 'Adaptations', 'Rugs', 'Almanacs', 'Communism', 'Inns & Hostels', 'Opposites', 'Shintoism', 'Sustainable Agriculture', 'Comparative Politics', 'Editing & Proofreading', 'Law', 'Bilingual Education', 'Presidents & First Families (U.S.)', 'Death', 'Customs', 'Nursery Rhymes', 'Devotional', 'Christmas & Advent', 'Tigers', 'Prophecy', 'Baby Food', 'Comedy', 'Board', 'Storage & Retrieval', 'Environmental', 'Skiing', 'Aesthetics', 'Metal Work', 'Camping & Outdoor Activities', 'Fertility & Infertility', 'Topology', 'Composition & Creative Writing', 'Infants & Toddlers', 'Framing', 'Pregnancy & Childbirth', 'Programming', 'Coins', 'Islamic Studies', 'ESP (Clairvoyance', 'Game', 'Crosswords', 'Agribusiness', 'Mystery & Detective', 'Holiday', 'Dysfunctional Families', 'Energy Policy', 'Strategic Planning', 'South', '21st Century', 'Apes', 'Parenting', 'Jesus', 'Depression', 'MD', 'Commercial', 'The Message', 'Decorating & Furnishings', 'Astronautics & Space Science', 'Medical Assistants', 'International Relations', 'Genres & Styles', 'Frogs & Toads', 'Topic', 'Western States', 'Phonetics & Phonology', 'Revolutionary', 'ID', 'Wrestling', 'Technology', 'Ancient', 'Wood Toys', 'Social Media', 'Holidays & Celebrations', 'Social Aspects', 'Dolls & Puppets', 'Educational Policy & Reform', 'Home & Community Care', 'Babysitting', 'Fishing', 'Jewish & Kosher', 'Immunology', 'Psychopathology', 'Human Services', 'Greek', 'Sexuality & Gender Studies', 'Christian Living', 'Kabbalah & Mysticism', 'Product', 'TN)', 'Public Affairs & Administration', 'West North Central (IA', 'Birds', 'Colors', 'Action & Adventure', 'Scrapbooking', 'Mariology', 'Professional', 'Monasticism', 'Journalism', 'Online Trading', 'King James Version', 'Disabilities & Special Needs', 'Dark Fantasy', 'Management Science', 'TN', 'Governmental', 'Later Years', 'Bed & Breakfast', 'Stepparenting', 'Vegan', 'East', 'Triathlon', 'Short Stories', 'Space Opera', 'Arthurian', 'Grief', 'New Business Enterprises', 'Wars & Conflicts (Other)', 'Architectural & Industrial', 'West Indies)', 'Hospital Administration & Care', 'School & Education', 'Amish', 'Rhetoric', 'Word Lists', 'Mobile & Wireless Communications', 'HTML', 'Java', 'Gifted', 'Ontario (ON)', 'Social Themes', 'Museums', 'Chinese', 'Fundamentalism', 'Irish', 'Holiday & Seasonal', 'Portraits', 'KS', 'Easter & Lent', 'Industrial & Organizational Psychology', 'Eating Disorders', 'Pacific (AK', 'Mechanical', 'Pneumatology', 'Rheumatology', 'Middle Eastern', 'Entertainment & Performing Arts', 'Entertainment & Games', 'Politics', 'Vocabulary', 'Mathematical Physics', 'Alternative Medicine', 'Violence in Society', 'Love & Marriage', 'State & Local', 'Medical (incl. Patients)', 'Kenya', 'Polar Regions', 'Other English Translations', 'Breastfeeding', 'Arithmetic', 'Folk & Traditional', 'Global Warming & Climate Change', 'Halloween', 'Crime & Mystery', 'Atlases', 'Anthologies (multiple authors)', 'Toys', 'Spiritual Warfare', 'Paralegals & Paralegalism', 'NH', 'Ethical & Legal Issues', 'Forecasting', 'Epic', 'Good & Evil', 'Gambling', 'Philanthropy & Charity', 'Environmental Science', 'Jewelry', 'Fire Science', 'Motorcycles', 'Biblical Stories & Studies', 'Sports Psychology', 'Office Automation', 'Mysteries & Detective Stories', 'Inclusive Education', 'Speech', 'Christianity', 'Native Americans', 'User Interfaces', 'Care & Restoration', 'Shaker', 'Information Theory', 'Contemporary Women', 'Teddy Bears & Stuffed Animals', 'Biblical Studies', 'Allergies', 'Holidays', 'Books & Libraries', 'Epistemology', 'Eldercare', 'Bears', 'Traditional British', 'Quizzes', 'Typography', 'Folkcrafts', 'Slow Cooking', 'Western', 'Origami', 'Horses', 'Lifestyles', 'Polarity)', 'Psychotherapy', 'Boxing', 'South Atlantic (DC', 'Alcoholism', 'Grandparenting', 'Botany', 'Maternity', 'Brunch & Tea Time', 'Lutheran', 'Feminism & Feminist Theory', 'Genetics', 'Ethics', 'Pain Medicine', 'Environmental Conservation & Protection', 'Date & Time', 'Personal Hygiene', 'Fitness & Exercise', 'Clothing & Dress', 'Comics', 'Media Studies', 'Japanese Gardens', 'Trivia', 'Midwest', 'Computer & Internet', 'Extended Family', 'Discipleship', 'Juices & Smoothies', 'France', 'Number Systems', 'Fractions', 'Intelligence & Espionage', 'Conservatism & Liberalism', 'Immune & Autoimmune', 'Holistic Medicine', 'Mountains', 'Job Hunting', 'Television: National Geographic', 'Time', 'Christian Science', 'Sailing', 'Bullying', 'Translating & Interpreting', 'Firearms & Weapons', 'Spices', 'Graphic Arts', 'Non-Religious', 'Conflict Resolution', 'Dogs', 'PSAT & NMSQT (National Merit Scholarship Qualifying Test)', 'Family Life', 'Trigonometry', 'General Practice', 'Social', 'Egypt', 'Coffee & Tea', 'Industrial Design', 'Rituals & Practice', 'Presidents & Heads of State', 'Money', 'Science', 'Cryptography', 'Compulsive Behavior', 'Recording & Reproduction', 'Decoration & Ornament', 'California Style', 'Elections', 'Catechisms', 'Suspense', 'Travel', 'Graphic Methods', 'Volunteer Work', 'Southeast', 'Scuba & Snorkeling', 'Mobile Devices', 'Anecdotes & Quotations', 'Image Processing', 'Survival Stories', 'Microwave', 'Cartography', 'Word Processing', 'HI', 'Pasta', 'Cell Biology', 'Physical Therapy', 'Public Relations', 'Communicative Disorders', 'Mysticism', 'Geese', 'Popular', 'Technological', 'Greenhouses', 'Woodworking', 'Hard Science Fiction', 'Pizza', 'Differential Equations', 'Christian Church', 'Elephants', 'Prayer & Spiritual', 'Channeling & Mediumship', 'Number Theory', 'Relationships', 'Biblical Criticism & Interpretation', 'Organized Crime', 'Ice & Figure Skating', 'System Administration', 'Retirement Planning', 'Children', 'Rice & Grains', 'Legal Profession', 'Rugby', 'Musculoskeletal', 'Low Cholesterol', 'Beer', 'Behaviorism', 'Pencil Drawing', 'Dye', 'Management Information Systems', 'Adolescent', 'Adventure & Adventurers', 'Diets', 'Literature & the Arts', 'Adulthood & Aging', 'Protestant', 'Ophthalmology', 'Juvenile Fiction', 'Thesauri', 'Operating Systems', 'Humor', 'VA', 'Devotional & Prayer', 'Korea', 'Body Art & Tattooing', 'Crafts for Children', 'Furniture & Cabinetry', 'World War I', 'Christology', 'Spiritualism', 'Lakes', 'Hardware', 'CA', 'Baseball', 'Writing Skills', 'NJ', 'Repair', 'XML', 'Ecology', 'Southern States', 'Masonry', 'Wildflowers', 'SAT', 'Portuguese', 'Shrubs', 'Literary', 'Toxicology', 'Model Railroading', 'Television & Radio', 'Roses', 'Writing', 'Yoga', 'Navigation', 'Grammar', 'Magick Studies', 'Food Science', 'Miniatures', 'Judaism', 'Perinatology & Neonatology', 'Monologues & Scenes', 'Alternative & Complementary Medicine', 'Food Content Guides', 'Information Management', 'Campaigns & Elections', 'Meetings & Presentations', 'Vietnamese', 'Regional & Ethnic', 'IL', 'Education & Training', 'Parent Participation', 'Railroads', 'Breeds', 'Vocabulary & Spelling', 'Discoveries', 'Skills', 'Knitting', 'Contemporary (1945-)', 'Desktop Publishing', 'Roller & In-Line Skating', 'Kwanzaa', 'Anxieties & Phobias', 'Couples & Family', 'Monsters', 'Scientology', 'Holocaust', 'Cleaning & Caretaking', 'Geography', 'Broadband', 'Erotica', 'Internet Marketing', 'Humorous', 'Liability', 'Environmental Policy', 'Cognitive Neuroscience & Cognitive Neuropsychology', 'Taoism', 'Wolves', 'Biophysics', 'Multi-Language Phrasebooks', 'Alligators & Crocodiles', 'Wildlife', 'Plumbing', 'Child & Adolescent', 'Railroads & Trains', 'School Safety', 'Sea Stories', 'Civil Procedure', 'Aims & Objectives', 'Pregnancy', 'Preaching', 'Denominations', 'Posters', 'Operations Research', 'Philosophy', 'Infectious Diseases', 'Rationalism', 'Institutions & Organizations', 'Pediatrics', 'Role Playing & Fantasy', 'Czech', 'Arts & Humanities', 'Classic & Allegory', 'Nutrition', 'Korean', 'Demography', 'Spiders', 'Geometry', 'Reference', 'Zen', 'Autographs', 'Biology', 'Wok', 'PCs', 'Developmental & Intellectual Disabilities', 'Black Humor']
X = data[features]
y = data['Price']

# Creating a copy of the DataFrame and dropping rows with NaN values
X_clean = X.dropna().copy()
y_clean = y[X_clean.index]  # Adjust the target variable accordingly if rows are dropped in X

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust parameters here

# Train the model
random_forest.fit(X_train, y_train)

# Predict on the test set
y_pred = random_forest.predict(X_test)

# Calculate evaluation metrics (e.g., Mean Squared Error, R-squared)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Get feature importances
feature_importance = random_forest.feature_importances_

# Match feature importances with their corresponding column names
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print or visualize the feature importances
print(feature_importance_df)

Mean Squared Error: 79.29139838230331
R-squared: -0.841694034231433
                                    Feature    Importance
0                                      Year  5.491181e-01
1                                     Month  3.288487e-01
1640                                    nan  8.999871e-03
631                                 Fiction  7.150861e-03
848                                 General  6.661927e-03
...                                     ...           ...
364                       Crisis Management  0.000000e+00
1577                            Orthopedics  0.000000e+00
109                    Commentary & Opinion  0.000000e+00
723                                   Sleep  0.000000e+00
1118  MCAT (Medical College Admission Test) -2.641372e-22

[2107 rows x 2 columns]
